# Índice de Entradas

1. [Timestamp Eliminator](#1)
2. [Curl File Generator](#2)
3. [Moduli Curl File Generator](#3)
4. [General Velocity File Generator](#4)
5. [Velocity Generator by Triad](#5)
6. [Vorticity File Generator](#6)
7. [Helicity Generator by Triads](#7)
8. [First Integral of Helicity Generator](#8)
9. [First Integral of Absolute Values of Helicity Generator](#9)
10. [General Enstrophy File Generator](#10)
11. [First Integral of Enstrophy](#11)
12. [First Integral Moduli Baseline vs Experimental Color](#12)
13. [Second Integral of Absolute Helicity Baseline vs Experimental Color](#13)


Este primer código se encarga de determinar la ruta en la que se encuentran los archivos que se van a trabajar

In [2]:

root_folder = r"F:\Datos\16Sep24"

# Horarios ajustados de mediciones de línea base y de intervenciones experimentales

# Horarios de mediciones de línea base
lab_b_matutino_base_times = {
    '01': "10:39:59.065",
    '02': "11:19:56.844",
    '03': "12:00:00.896",
    '04': "12:39:59.139",
    '05': "13:19:59.461"
}

# Horarios de intervenciones experimentales
lab_b_matutino_intervention_times = {
    '01': "10:59:59.414",
    '02': "11:39:59.104",
    '03': "12:19:59.778",
    '04': "13:00:01.295"
}

# Horarios de mediciones de línea base para Lab_A en turno matutino
lab_a_matutino_base_times = {
    '01': "10:39:59.065",
    '02': "11:19:56.844",
    '03': "12:00:00.896",
    '04': "12:39:59.139",
    '05': "13:19:59.461"
}

# Horarios de intervenciones experimentales para Lab_A en turno matutino
lab_a_matutino_intervention_times = {
    '01': "10:59:59.414",
    '02': "11:39:59.104",
    '03': "12:19:59.778",
    '04': "13:00:01.295"
}

n_change = {"Lab_Betta":"Lab_Betta", "Lab_Gamma":"Lab_Gamma"}

triad_names = {
    "FRT": "Frontal-Derecho-Superior",  
    "PLB": "Trasero-Izquierdo-Inferior",  
    "FLT": "Frontal-Izquierdo-Superior",  
    "PRB": "Trasero-Derecho-Inferior",  
    "FRB": "Frontal-Derecho-Inferior",  
    "PLT": "Trasero-Izquierdo-Superior",  
    "FLB": "Frontal-Izquierdo-Inferior",  
    "PRT": "Trasero-Derecho-Superior",  
    "RTB": "Derecho-Superior-Inferior",  
    "FLP": "Frontal-Izquierdo-Trasero",  
    "LTB": "Izquierdo-Superior-Inferior",  
    "FRP": "Frontal-Derecho-Trasero"
}

## 1

$$

    \Huge \text{Second Timestamp Eliminator}

$$


In [ ]:
import os
import re

# Función para actualizar el formato de las líneas
def update_file_format(filepath):
    updated_lines = []
    print(f"Actualizando formato del archivo: {filepath}")
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            print(f"Procesando línea: {line.strip()}")  # Mostrar cada línea procesada
            # Verificar si la línea contiene dos estampas de tiempo
            if re.match(r'^\d{2}:\d{2}:\d{2}\.\d{3} -> \d{2}:\d{2}:\d{2}\.\d{3} -> ', line):
                # Extraer y conservar sólo desde la segunda estampa de tiempo hasta el final de la línea
                new_line = re.sub(r'^\d{2}:\d{2}:\d{2}\.\d{3} -> ', '', line.strip())
                print(f"Formato actualizado: {new_line}")  # Mostrar la línea actualizada
                updated_lines.append(new_line)
            else:
                updated_lines.append(line.strip())
    
    # Sobrescribir el archivo con el nuevo formato
    with open(filepath, 'w', encoding='utf-8') as file:
        for line in updated_lines:
            file.write(line + '\n')
    print(f"Formato actualizado en: {filepath}")

# Función para eliminar filas con valores menores a 3.0
def filter_file(filepath):
    filtered_lines = []
    print(f"Filtrando archivo: {filepath}")
    with open(filepath, 'r', encoding='utf-8') as infile:
        for line in infile:
            # Buscar el vector entre paréntesis
            match = re.search(r'\((.*?)\)', line)
            if match:
                # Convertir los valores del vector a float
                values = list(map(float, match.group(1).split(',')))
                # Si todos los valores son mayores o iguales a 3.0, conservar la línea
                if all(value >= 3.0 for value in values):
                    filtered_lines.append(line.strip())
                else:
                    print(f"Línea eliminada: {line.strip()}")  # Mostrar la línea eliminada
            else:
                filtered_lines.append(line.strip())

    # Sobrescribir el archivo con las líneas filtradas
    with open(filepath, 'w', encoding='utf-8') as outfile:
        for line in filtered_lines:
            outfile.write(line + '\n')
    print(f"Archivo filtrado guardado: {filepath}")

# Función para escanear un directorio y procesar archivos con sufijos específicos
def scan_directory(root_folder, file_suffixes):
    for root, dirs, files in os.walk(root_folder):
        dirs[:] = [d for d in dirs if d not in ["Data Analysis", "Programas"]]  # Excluir directorios
        for filename in files:
            if any(filename.endswith(suffix) for suffix in file_suffixes):
                file_path = os.path.join(root, filename)
                update_file_format(file_path)  # Actualizar formato del archivo
                filter_file(file_path)  # Filtrar el archivo

file_suffixes = ["medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt", "mednegra.txt", "mednaranja.txt"]

# Realiza la búsqueda y procesa los archivos
scan_directory(root_folder, file_suffixes)

## 2

$$

    \Huge \text{Curl File Generator}

$$



In [ ]:
#Rotacional .txt
import numpy as np
import os
from datetime import datetime, timedelta
import re

def read_sensor_data(filepath):
    data = []
    timestamps = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])
                    numbers = parts[1].strip('()').split(', ')
                    data.append([float(num) for num in numbers])
        return timestamps, np.array(data)
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
        return [], np.array([])

def calculate_curl(data):
    try:
        Dx = Dy = Dz = 1.0
        curl_x = (data[:, 1] - data[:, 0]) / Dx # (Trasero - Frontal) / Dx
        curl_y = (data[:, 3] - data[:, 2]) / Dy # (Derecha - Izquierda) / Dy
        curl_z = (data[:, 5] - data[:, 4]) / Dz # (Inferior - Superior) / Dz
        return np.array([curl_x, curl_y, curl_z]).T
    except Exception as e:
        print(f"Error calculating curl: {e}")
        return np.array([])

def adjust_timestamps(base_time, timestamps):
    try:
        base_datetime = datetime.strptime(base_time, '%H:%M:%S.%f')
        adjusted_timestamps = []
        for ts in timestamps:
            time_delta = datetime.strptime(ts, '%H:%M:%S.%f') - datetime.strptime('00:00:00.000', '%H:%M:%S.%f')
            new_datetime = base_datetime + time_delta
            adjusted_timestamps.append(new_datetime.strftime('%H:%M:%S.%f')[:-3])
        return adjusted_timestamps
    except Exception as e:
        print(f"Error adjusting timestamps: {e}")
        return []

def find_experiment_folder(root_folder, lab, shift):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder)
    return None

def process_files(input_folder, output_folder, base_times, file_suffixes=None):
    generated_files = []
    for root, dirs, files in os.walk(input_folder):
        dirs[:] = [d for d in dirs if "Data Analysis" not in os.path.join(root, d)]
        for filename in files:
            if "00" in filename:
                continue
            if file_suffixes:
                if not any(filename.endswith(suffix) for suffix in file_suffixes):
                    continue
            else:
                if not filename.endswith('mednegra.txt'):
                    continue
                
            file_path = os.path.join(root, filename)
            try:
                timestamps, data = read_sensor_data(file_path)
                if data.size == 0:
                    continue
                curls = calculate_curl(data)
                if curls.size == 0:
                    continue
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
                continue
            
            file_number = filename.split('mednegra.txt')[0][-2:] if not file_suffixes else filename[:2]
            adjusted_timestamps = adjust_timestamps(base_times.get(file_number, "00:00:00.000"), timestamps)

            output_filename = f"curl_{filename}"
            output_file_path = os.path.join(output_folder, output_filename)
            try:
                with open(output_file_path, "w") as f:
                    for i, curl in enumerate(curls):
                        f.write(f"{adjusted_timestamps[i]} -> ({curl[0]:.10f}, {curl[1]:.10f}, {curl[2]:.10f})\n")
                generated_files.append(output_filename)
            except Exception as e:
                print(f"Error writing file {output_file_path}: {e}")

    return generated_files

def process_lab(root_folder, lab_name, base_times, intervention_times):
    for shift in ['V', 'M']:
        shift_name = "Vespertino" if shift == 'V' else "Matutino"
        experiment_folder = find_experiment_folder(root_folder, lab_name, shift)
        if not experiment_folder:
            print(f"No experiment folder found for {lab_name}, {shift_name} shift.")
            continue
        
        print(f"Processing directory: {experiment_folder}")

        # Procesar la línea base
        output_folder = os.path.join(experiment_folder, "Data Analysis", "Processing Data", "Curl_Baseline")
        os.makedirs(output_folder, exist_ok=True)
        generated_files = process_files(experiment_folder, output_folder, base_times)
        
        print(f"Generated files for {lab_name}, {shift_name} shift Baseline:")
        for file in generated_files:
            print(f"  - {file}")

        # Procesar las intervenciones
        output_folder = os.path.join(experiment_folder, "Data Analysis", "Processing Data", "Curl_Experimental_Color")
        os.makedirs(output_folder, exist_ok=True)
        file_suffixes = ["medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt"]
        generated_files = process_files(experiment_folder, output_folder, intervention_times, file_suffixes)
        
        print(f"Generated files for {lab_name}, {shift_name} shift Experimental:")
        for file in generated_files:
            print(f"  - {file}")



# Procesar todos los laboratorios y turnos
labs = {
    "Lab_Betta": {
        "base_times": lab_a_matutino_base_times,
        "intervention_times": lab_a_matutino_intervention_times,
    }  ,
    "Lab_Gamma": {
        "base_times": lab_b_matutino_base_times,
        "intervention_times": lab_b_matutino_intervention_times,
    }
}



for lab_name, times in labs.items():
    process_lab(root_folder, lab_name, times["base_times"], times["intervention_times"])


## 3

$$

    \Huge \text{Moduli Curl File Generator}

$$



In [ ]:
#Magnitud rotacional.txt
import numpy as np
import os
from datetime import datetime, timedelta
import re

def read_sensor_data(filepath):
    data = []
    timestamps = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                timestamps.append(parts[0])
                numbers = parts[1].strip('()').split(', ')
                data.append([float(num) for num in numbers])
    return timestamps, np.array(data)

def calculate_magnitude(data):
    return np.linalg.norm(data, axis=1)

def adjust_timestamps(base_time, timestamps):
    base_datetime = datetime.strptime(base_time, '%H:%M:%S.%f')
    adjusted_timestamps = []
    for ts in timestamps:
        time_delta = datetime.strptime(ts, '%H:%M:%S.%f') - datetime.strptime('00:00:00.000', '%H:%M:%S.%f')
        new_datetime = base_datetime + time_delta
        adjusted_timestamps.append(new_datetime.strftime('%H:%M:%S.%f')[:-3])
    return adjusted_timestamps

def process_files(input_folder, output_folder, base_times, file_suffixes=None):
    generated_files = []  # Lista para almacenar los nombres de los archivos generados
    for root, dirs, files in os.walk(input_folder):
        dirs[:] = [d for d in dirs if "Data Analysis" not in os.path.join(root, d)]  # Excluir directorios que contienen 'Data Analysis'
        for filename in files:
            if "00" in filename:  # Omitir archivos con "00" en el nombre
                continue
            if file_suffixes:
                if not any(filename.endswith(suffix) for suffix in file_suffixes):
                    continue
            else:
                if not filename.endswith('mednegra.txt'):
                    continue
                
            file_path = os.path.join(root, filename)
            try:
                timestamps, data = read_sensor_data(file_path)
                magnitudes = calculate_magnitude(data)
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
                continue
            
            file_number = filename.split('mednegra.txt')[0][-2:] if not file_suffixes else filename[:2]
            adjusted_timestamps = adjust_timestamps(base_times.get(file_number, "00:00:00.000"), timestamps)

            magnitude_filename = f"magnitude_{filename}"
            magnitude_file_path = os.path.join(output_folder, magnitude_filename)
            try:
                with open(magnitude_file_path, "w") as f:
                    for i, magnitude in enumerate(magnitudes):
                        f.write(f"{adjusted_timestamps[i]} -> {magnitude:.10f}\n")
                generated_files.append(magnitude_filename)  # Agregar el nombre del archivo generado a la lista
            except Exception as e:
                print(f"Error writing file {magnitude_file_path}: {e}")

    return generated_files  # Devolver la lista de archivos generados

def find_experiment_folder(root_folder, lab, shift):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder), date_regex.search(folder).group(0)
    return None, None

def process_lab(root_folder, lab_name, base_times, intervention_times):
    for shift in ['V', 'M']:
        shift_name = "Vespertino" if shift == 'V' else "Matutino"
        experiment_folder, date_str = find_experiment_folder(root_folder, lab_name, shift)
        if not experiment_folder:
            print(f"No se encontró ningún directorio para {lab_name}, {shift}.")
            continue

        input_folder = experiment_folder
        if not os.path.exists(input_folder):
            continue
        
        # Procesar la línea base
        output_folder = os.path.join(input_folder, "Data Analysis", "Processing Data", "Moduli_Curl_Baseline")
        os.makedirs(output_folder, exist_ok=True)
        generated_files = process_files(input_folder, output_folder, base_times)
        
        print(f"Generated files for {lab_name}, {shift_name} shift Baseline:")
        for file in generated_files:
            print(f"  - {file}")

        # Procesar las intervenciones
        output_folder = os.path.join(input_folder, "Data Analysis", "Processing Data", "Moduli_Curl_Experimental_Color")
        os.makedirs(output_folder, exist_ok=True)
        file_suffixes = ["medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt", "mednaranja.txt"]
        generated_files = process_files(input_folder, output_folder, intervention_times, file_suffixes)
        
        print(f"Generated files for {lab_name}, {shift_name} shift Experimental:")
        for file in generated_files:
            print(f"  - {file}")


# Procesar todos los laboratorios y turnos
labs = {
    "Lab_Betta": {
        "base_times": lab_a_matutino_base_times,
        "intervention_times": lab_a_matutino_intervention_times,
    }  
,
    "Lab_Gamma": {
        "base_times": lab_b_matutino_base_times,
        "intervention_times": lab_b_matutino_intervention_times,
    }
}

for lab_name, times in labs.items():
    process_lab(root_folder, lab_name, times["base_times"], times["intervention_times"])


## 4

$$

    \Huge \text{General Velocity File Generator}

$$


In [ ]:
#Código requsito previo para correr el código de las 12 triadas de velocidad-------------- Velocidad.txt (no triadas)
import numpy as np
import os
from datetime import datetime, timedelta
import re

def read_sensor_data(filepath):
    data = []
    timestamps = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])  # Guardar marca de tiempo
                    numbers = parts[1].strip('()').split(', ')
                    data.append([float(num) for num in numbers])
        return timestamps, np.array(data)
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
        return [], np.array([])

def calculate_velocities(data, delta_t):
    # Calcular velocidades usando diferencias hacia adelante
    velocities = (data[1:] - data[:-1]) / delta_t
    return velocities

def adjust_timestamps(base_time, timestamps):
    try:
        base_datetime = datetime.strptime(base_time, '%H:%M:%S.%f')
        adjusted_timestamps = []
        for ts in timestamps[:-1]:  # Excluir la última marca de tiempo
            current_time = datetime.strptime(ts, '%H:%M:%S.%f')
            adjusted_time = base_datetime + (current_time - datetime.strptime(timestamps[0], '%H:%M:%S.%f'))
            adjusted_timestamps.append(adjusted_time.strftime('%H:%M:%S.%f')[:-3])
        return adjusted_timestamps
    except Exception as e:
        print(f"Error adjusting timestamps: {e}")
        return []

def find_experiment_folder(root_folder, lab, shift):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder), date_regex.search(folder).group(0)
    return None, None

def process_files(input_folder, output_folder, delta_t, base_times, file_suffixes=None):
    os.makedirs(output_folder, exist_ok=True)  # Asegurar que el directorio de salida exista
    generated_files = []  # Lista para almacenar los nombres de los archivos generados

    for root, dirs, files in os.walk(input_folder):
        dirs[:] = [d for d in dirs if "Data Analysis" not in os.path.join(root, d)]  # Excluir directorios que contienen 'Data Analysis'
        for filename in files:
            if "00" in filename:  # Omitir archivos con "00" en el nombre
                continue
            if file_suffixes:
                if not any(filename.endswith(suffix) for suffix in file_suffixes):
                    continue
            else:
                if not filename.endswith('mednegra.txt'):
                    continue
                
            file_path = os.path.join(root, filename)
            try:
                timestamps, data = read_sensor_data(file_path)
                if data.size == 0:
                    continue
                velocities = calculate_velocities(data, delta_t)
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
                continue
            
            file_number = filename.split('mednegra.txt')[0][-2:] if not file_suffixes else filename[:2]
            adjusted_timestamps = adjust_timestamps(base_times.get(file_number, "00:00:00.000"), timestamps)

            output_filename = f"velocity_{filename}"
            output_file_path = os.path.join(output_folder, output_filename)
            try:
                with open(output_file_path, "w") as f:
                    for i, velocity in enumerate(velocities):
                        f.write(f"{adjusted_timestamps[i]} -> ({velocity[0]:.10f}, {velocity[1]:.10f}, {velocity[2]:.10f}, {velocity[3]:.10f}, {velocity[4]:.10f}, {velocity[5]:.10f})\n")
                generated_files.append(output_filename)  # Agregar el nombre del archivo generado a la lista
            except Exception as e:
                print(f"Error writing file {output_file_path}: {e}")

    return generated_files  # Devolver la lista de archivos generados

def process_lab(root_folder, lab_name, base_times, intervention_times, delta_t):
    for shift in ['V', 'M']:
        shift_name = "Vespertino" if shift == 'V' else "Matutino"
        experiment_folder, date_str = find_experiment_folder(root_folder, lab_name, shift)
        if not experiment_folder:
            print(f"No se encontró ningún directorio para {lab_name}, {shift}.")
            continue
        
        # Procesar la línea base
        output_folder = os.path.join(experiment_folder, "Data Analysis", "Processing Data", "Velocity_Baseline")
        os.makedirs(output_folder, exist_ok=True)
        generated_files = process_files(experiment_folder, output_folder, delta_t, base_times)
        
        print(f"Generated files for {lab_name}, {shift_name} shift Baseline:")
        for file in generated_files:
            print(f"  - {file}")

        # Procesar las intervenciones
        output_folder = os.path.join(experiment_folder, "Data Analysis", "Processing Data", "Velocity_Experimental_Color")
        os.makedirs(output_folder, exist_ok=True)
        file_suffixes = ["medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt"]
        generated_files = process_files(experiment_folder, output_folder, delta_t, intervention_times, file_suffixes)
        
        print(f"Generated files for {lab_name}, {shift_name} shift Experimental:")
        for file in generated_files:
            print(f"  - {file}")



# Procesar todos los laboratorios y turnos
labs = {
    "Lab_Betta": {
        "base_times": lab_a_matutino_base_times,
        "intervention_times": lab_a_matutino_intervention_times,
    }  
,
    "Lab_Gamma": {
        "base_times": lab_b_matutino_base_times,
        "intervention_times": lab_b_matutino_intervention_times,
    }

}
delta_t = 0.01  # Paso de tiempo para el cálculo de velocidades

for lab_name, times in labs.items():
    process_lab(root_folder, lab_name, times["base_times"], times["intervention_times"], delta_t)


## 5

$$

    \Huge \text{Velocity Generator by Triad}

$$


In [ ]:
#Calculo de velocidades por triada .txt
import os

def extract_velocity_components(root_folder):
    generated_files = []  # Lista para almacenar los nombres de los archivos generados
    
    triad_codes = {
        (0, 2, 4): "FRT",  
        (1, 3, 5): "PLB",  
        (0, 3, 4): "FLT",  
        (1, 2, 5): "PRB",  
        (0, 2, 5): "FRB",  
        (1, 3, 4): "PLT",  
        (0, 3, 5): "FLB",  
        (1, 2, 4): "PRT",  
        (2, 4, 5): "RTB",  
        (0, 1, 3): "FLP",  
        (3, 4, 5): "LTB",  
        (0, 1, 2): "FRP"
    }
    
    for root, dirs, files in os.walk(root_folder):
        if "Velocity_Baseline" in root or "Velocity_Experimental_Color" in root:
            # Usar la carpeta actual, sin agregar '_Component'
            new_folder = root
            
            # Iterar sobre cada archivo en la carpeta original
            for filename in files:
                if filename.startswith("v") and filename.endswith(".txt"):
                    path = os.path.join(root, filename)
                    
                    with open(path, 'r') as file:
                        file_content = file.readlines()  # Leer todo el contenido una vez
                        
                        # Generar archivos por cada triada y usando su código del diccionario
                        for triad, code in triad_codes.items():
                            triad_folder = os.path.join(new_folder, code)
                            os.makedirs(triad_folder, exist_ok=True)  # Crear la carpeta de la triada si no existe
                            new_path = os.path.join(triad_folder, f"Velocity_{code}_" + filename)
                            
                            with open(new_path, 'w') as triad_file:
                                for line in file_content:
                                    time_stamp, velocities = line.strip().split(' -> ')
                                    v = eval(velocities)
                                    selected_components = tuple(v[j] for j in triad)
                                    triad_file.write(f"{time_stamp} -> {selected_components}\n")
                            
                            # Almacenar solo el nombre de la carpeta de cabecera y los nombres de los archivos generados
                            header_folder = os.path.basename(triad_folder)
                            file_name = os.path.basename(new_path)
                            generated_files.append(f"{header_folder}/{file_name}")

    return generated_files  # Devolver la lista de archivos generados

# La ruta base donde están las carpetas de origen
generated_files = extract_velocity_components(root_folder)

# Imprimir los nombres de los archivos generados
print("Generated files:")
for file in generated_files:
    print(f"  - {file}")

## 6

$$

    \Huge \text{Vorticity File Generator}

$$


In [ ]:
#Vorticidad .txt
import os
import numpy as np
import re

def read_velocity_data(filepath):
    data = []
    timestamps = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])  # Guardar timestamps directamente del archivo
                    velocity_components = parts[1].strip('()').split(', ')
                    if len(velocity_components) == 6:  # Asegurarse de que haya 6 componentes
                        data.append([float(v) for v in velocity_components])
        return timestamps, np.array(data)
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
        return [], np.array([])

def calculate_vorticity(velocities, indices):
    if velocities.ndim != 2 or velocities.shape[1] != 3:
        print(f"velocities.ndim: {velocities.ndim}, velocities.shape: {velocities.shape}")
        raise ValueError("El array de velocidades debe ser bidimensional con tres columnas.")
    
    Vx, Vy, Vz = indices
    dvz_dy = np.gradient(velocities[:, Vz], axis=0)
    dvy_dz = np.gradient(velocities[:, Vy], axis=0)
    dvx_dz = np.gradient(velocities[:, Vx], axis=0)
    dvz_dx = np.gradient(velocities[:, Vz], axis=0)
    dvy_dx = np.gradient(velocities[:, Vy], axis=0)
    dvx_dy = np.gradient(velocities[:, Vx], axis=0)

    omega_x = dvz_dy - dvy_dz
    omega_y = dvx_dz - dvz_dx
    omega_z = dvy_dx - dvx_dy

    return np.array([omega_x, omega_y, omega_z]).T

def save_vorticity_with_timestamps(timestamps, vorticities, filename):
    os.makedirs(os.path.dirname(filename), exist_ok=True)  # Crear el directorio si no existe
    print(f"Saving vorticity data to: {filename}")
    
    try:
        with open(filename, 'w') as f:
            for ts, vort in zip(timestamps, vorticities):
                formatted_output = f"{ts} -> ({vort[0]:.10f}, {vort[1]:.10f}, {vort[2]:.10f})\n"
                f.write(formatted_output)
    except Exception as e:
        print(f"Error writing file {filename}: {e}")

def process_files(input_folder, output_folder, file_suffixes, triads):
    if not os.path.exists(input_folder):
        print(f"Input folder does not exist: {input_folder}")
        return []
    
    generated_files = []  # Lista para almacenar los nombres de los archivos generados
    
    for filename in os.listdir(input_folder):
        if any(filename.endswith(suffix) for suffix in file_suffixes):
            filepath = os.path.join(input_folder, filename)
            print(f"Processing file: {filepath}")
            timestamps, velocities = read_velocity_data(filepath)
            
            if velocities.ndim != 2 or velocities.shape[1] != 6:
                print(f"El archivo {filename} no contiene datos de velocidad válidos. Dimensiones: {velocities.shape}")
                continue
            
            for triad in triads:
                if triad in triad_codes:
                    code = triad_codes[triad]
                    triad_folder = os.path.join(output_folder, code)
                    os.makedirs(triad_folder, exist_ok=True)  # Crear carpeta para la triada si no existe
                    velocities_triad = velocities[:, triad]
                    
                    print(f"Calculating vorticity for triad {code}: {filename}")
                    vorticity = calculate_vorticity(velocities_triad, [0, 1, 2])
                    
                    output_file = f"vorticity_{code}_{filename}"
                    save_vorticity_with_timestamps(timestamps, vorticity, os.path.join(triad_folder, output_file))
                    
                    generated_files.append(f"{os.path.basename(triad_folder)}/{output_file}")

    return generated_files  # Devolver la lista de archivos generados

def find_experiment_folder(root_folder, lab, shift):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder), date_regex.search(folder).group(0)
    return None, None

def process_lab(root_folder, lab_name, file_suffixes, triads):
    for shift in ['V', 'M']:
        shift_name = "Vespertino" if shift == 'V' else "Matutino"
        experiment_folder, date_str = find_experiment_folder(root_folder, lab_name, shift)
        if not experiment_folder:
            print(f"No se encontró ningún directorio para {lab_name}, {shift}.")
            continue

        input_folder_base = os.path.join(experiment_folder, "Data Analysis", "Processing Data")
        
        # Procesar la línea base
        input_folder = os.path.join(input_folder_base, "Velocity_Baseline")
        output_folder = os.path.join(input_folder_base, "Vorticity_Baseline")
        generated_files = process_files(input_folder, output_folder, file_suffixes, triads)
        print(f"Generated vorticity files for {lab_name}, {shift_name} shift Baseline:")
        for file in generated_files:
            print(f"  - {file}")

        # Procesar las intervenciones
        input_folder = os.path.join(input_folder_base, "Velocity_Experimental_Color")
        output_folder = os.path.join(input_folder_base, "Vorticity_Experimental_Color")
        generated_files = process_files(input_folder, output_folder, file_suffixes, triads)
        print(f"Generated vorticity files for {lab_name}, {shift_name} shift Experimental Color:")
        for file in generated_files:
            print(f"  - {file}")

# Define the root folder and suffixes
file_suffixes = ["medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt", "mednegra.txt"]

# Define the triads
triads = [
    (0, 2, 4),  # T1 (FRONTAL, DERECHA, SUPERIOR)
    (1, 3, 5),  # T2 (TRASERO, IZQUIERDA, INFERIOR)
    (0, 3, 4),  # T3 (FRONTAL, IZQUIERDA, SUPERIOR)
    (1, 2, 5),  # T4 (TRASERO, DERECHA, INFERIOR)
    (0, 2, 5),  # T5 (FRONTAL, DERECHA, INFERIOR)
    (1, 3, 4),  # T6 (TRASERO, IZQUIERDA, SUPERIOR)
    (0, 3, 5),  # T7 (FRONTAL, IZQUIERDA, INFERIOR)
    (1, 2, 4),  # T8 (TRASERO, DERECHA, SUPERIOR)
    (2, 4, 5),  # T9 (DERECHA, SUPERIOR, INFERIOR)
    (0, 1, 3),  # T10 (IZQUIERDA, FRONTAL, TRASERO)
    (3, 4, 5),  # T11 (IZQUIERDA, SUPERIOR, INFERIOR)
    (0, 1, 2)   # T12 (DERECHA, FRONTAL, TRASERO)
]

triad_codes = {
        (0, 2, 4): "FRT",  
        (1, 3, 5): "PLB",  
        (0, 3, 4): "FLT",  
        (1, 2, 5): "PRB",  
        (0, 2, 5): "FRB",  
        (1, 3, 4): "PLT",  
        (0, 3, 5): "FLB",  
        (1, 2, 4): "PRT",  
        (2, 4, 5): "RTB",  
        (0, 1, 3): "FLP",  
        (3, 4, 5): "LTB",  
        (0, 1, 2): "FRP"
    }

# Procesar todos los laboratorios y turnos
labs = ["Lab_Betta", "Lab_Gamma"]

for lab_name in labs:
    process_lab(root_folder, lab_name, file_suffixes, triads)


## 7

$$

    \Huge \text{Helicity Generator by Triads}

$$


In [ ]:
#Helicidad general .txt
import os
import numpy as np
import re

def read_velocity_data(filepath):
    data = []
    timestamps = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])
                    velocity_components = parts[1].strip('()').split(', ')
                    data.append([float(v) for v in velocity_components])
        return timestamps, np.array(data)
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
        return [], np.array([])

def calculate_helicity(velocities, vorticities, indices, volume):
    selected_velocities = velocities[:, indices]
    helicity = np.einsum('ij,ij->i', selected_velocities, vorticities) * volume
    return helicity

def format_and_save_helicities(timestamps, helicities, filename):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    try:
        with open(filename, "w") as file:
            for timestamp, helicity in zip(timestamps, helicities):
                formatted_output = f"{timestamp} -> {helicity:.10f}\n"
                file.write(formatted_output)
    except Exception as e:
        print(f"Error writing file {filename}: {e}")

def process_files(input_folder_velocity, input_folder_vorticity, output_folder, file_suffixes, volume, triads):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    generated_files = []
    
    for suffix in file_suffixes:
        for filename in os.listdir(input_folder_velocity):
            if filename.endswith(suffix):
                base_filename = filename.split('.')[0]
                filepath_velocity = os.path.join(input_folder_velocity, filename)
                
                for triad in triads:
                    if triad in triad_codes:
                        code = triad_codes[triad]
                        triad_folder = os.path.join(input_folder_vorticity, code)
                        filepath_vorticity = os.path.join(triad_folder, f"vorticity_{code}_{base_filename}.txt")
                        
                        timestamps, velocities = read_velocity_data(filepath_velocity)
                        if velocities.size == 0:
                            continue
                        
                        _, vorticity = read_velocity_data(filepath_vorticity)

                        if vorticity.size == 0:
                            print(f"Skipping file {filename} due to missing vorticity data for triad {code}.")
                            continue
                        
                        helicity = calculate_helicity(velocities, vorticity, triad, volume)
                        
                        output_filename = f"helicity_{code}_{base_filename}.txt"
                        output_path = os.path.join(output_folder, code, output_filename)
                        
                        format_and_save_helicities(timestamps, helicity, output_path)
                        
                        generated_files.append(f"{os.path.basename(output_folder)}/{code}/{output_filename}")

    return generated_files

def find_experiment_folder(root_folder, lab, shift):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder), date_regex.search(folder).group(0)
    return None, None

def process_lab(root_folder, lab_name, file_suffixes, volume, triads):
    for shift in ['V', 'M']:
        shift_name = "Vespertino" if shift == 'V' else "Matutino"
        experiment_folder, date_str = find_experiment_folder(root_folder, lab_name, shift)
        if not experiment_folder:
            print(f"No se encontró ningún directorio para {lab_name}, {shift}.")
            continue

        input_folder_base = os.path.join(experiment_folder, "Data Analysis", "Processing Data")
        
        if not os.path.exists(input_folder_base):
            print(f"Cannot access directory: {input_folder_base}")
            continue

        # Procesar la línea base
        input_folder_velocity = os.path.join(input_folder_base, "Velocity_Baseline")
        input_folder_vorticity = os.path.join(input_folder_base, "Vorticity_Baseline")
        output_folder = os.path.join(input_folder_base, "Helicity_Baseline")
        generated_files = process_files(input_folder_velocity, input_folder_vorticity, output_folder, file_suffixes, volume, triads)
        print(f"Generated helicity files for {lab_name}, {shift_name} shift Baseline:")
        for file in generated_files:
            print(f"  - {file}")

        # Procesar las intervenciones
        input_folder_velocity = os.path.join(input_folder_base, "Velocity_Experimental_Color")
        input_folder_vorticity = os.path.join(input_folder_base, "Vorticity_Experimental_Color")
        output_folder = os.path.join(input_folder_base, "Helicity_Experimental_Color")
        generated_files = process_files(input_folder_velocity, input_folder_vorticity, output_folder, file_suffixes, volume, triads)
        print(f"Generated helicity files for {lab_name}, {shift_name} shift Experimental Color:")
        for file in generated_files:
            print(f"  - {file}")

# Define the root folder and suffixes
file_suffixes = ["medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt", "mednegra.txt"]

# Volumes for each lab
volumes = {
    "Lab_Betta": 7.3195, # Volume for Lab A
    "Lab_Gamma": 6.8342  # Volume for Lab B (example)
}

# Triads
triads = [
    (0, 2, 4),  # T1 (FRONTAL, DERECHA, SUPERIOR)
    (1, 3, 5),  # T2 (TRASERO, IZQUIERDA, INFERIOR)
    (0, 3, 4),  # T3 (FRONTAL, IZQUIERDA, SUPERIOR)
    (1, 2, 5),  # T4 (TRASERO, DERECHA, INFERIOR)
    (0, 2, 5),  # T5 (FRONTAL, DERECHA, INFERIOR)
    (1, 3, 4),  # T6 (TRASERO, IZQUIERDA, SUPERIOR)
    (0, 3, 5),  # T7 (FRONTAL, IZQUIERDA, INFERIOR)
    (1, 2, 4),  # T8 (TRASERO, DERECHA, SUPERIOR)
    (2, 4, 5),  # T9 (DERECHA, SUPERIOR, INFERIOR)
    (0, 1, 3),  # T10 (IZQUIERDA, FRONTAL, TRASERO)
    (3, 4, 5),  # T11 (IZQUIERDA, SUPERIOR, INFERIOR)
    (0, 1, 2)   # T12 (DERECHA, FRONTAL, TRASERO)
]

# Triad codes
triad_codes = {
    (0, 2, 4): "FRT",  
    (1, 3, 5): "PLB",  
    (0, 3, 4): "FLT",  
    (1, 2, 5): "PRB",  
    (0, 2, 5): "FRB",  
    (1, 3, 4): "PLT",  
    (0, 3, 5): "FLB",  
    (1, 2, 4): "PRT",  
    (2, 4, 5): "RTB",  
    (0, 1, 3): "FLP",  
    (3, 4, 5): "LTB",  
    (0, 1, 2): "FRP"
}

# Procesar todos los laboratorios y turnos
labs = ["Lab_Betta", "Lab_Gamma"]

for lab_name in labs:
    process_lab(root_folder, lab_name, file_suffixes, volumes[lab_name], triads)


## 8

$$

    \Huge \text{First Integral of Helicity Generator}

$$


In [ ]:
#Integral Helicidad .txt
import os
import re

def process_file(file_path, output_file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    accumulated_helicity = 0
    results = []
    for line in lines:
        time_stamp, value = line.split('->')
        value = float(value.strip())
        accumulated_helicity += value
        results.append(f"{time_stamp.strip()} -> {accumulated_helicity:.10f}\n")

    with open(output_file_path, 'w') as output_file:
        output_file.writelines(results)

def generate_accumulated_helicity(input_directory, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for root, dirs, files in os.walk(input_directory):
        for filename in files:
            if filename.endswith(".txt"):
                file_path = os.path.join(root, filename)
                output_filename = f"sum_{filename}"
                output_file_path = os.path.join(output_directory, output_filename)
                process_file(file_path, output_file_path)
                print(f"Generated file: {output_filename}")

def find_experiment_folder(root_folder, lab, shift):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder), date_regex.search(folder).group(0)
    return None, None

def process_lab_helicity(root_folder, lab_name, shift, file_suffixes, triad_codes):
    shift_name = "Vespertino" if shift == 'V' else "Matutino"
    experiment_folder, date_str = find_experiment_folder(root_folder, lab_name, shift)
    if not experiment_folder:
        print(f"No se encontró ningún directorio para {lab_name}, {shift}.")
        return

    input_folder_base = os.path.join(experiment_folder, "Data Analysis", "Processing Data")
    
    if not os.path.exists(input_folder_base):
        print(f"Skipping {shift_name} shift for {lab_name} as directory does not exist: {input_folder_base}")
        return

    # Procesar los acumulados de Baseline para cada triada
    for triad, triad_code in triad_codes.items():
        input_directory = os.path.join(input_folder_base, "Helicity_Baseline", triad_code)
        output_directory = os.path.join(input_folder_base, "Sum_Helicity_Baseline", triad_code)
        if os.path.exists(input_directory):
            generate_accumulated_helicity(input_directory, output_directory)
            print(f"Generated accumulated helicity files for {lab_name}, {shift_name} shift Baseline, Triad {triad_code}.")
        else:
            print(f"Skipping {shift_name} shift for {lab_name} Baseline Triad {triad_code} as directory does not exist: {input_directory}")

    # Procesar los acumulados de Experimental Color para cada triada
    for triad, triad_code in triad_codes.items():
        input_directory = os.path.join(input_folder_base, "Helicity_Experimental_Color", triad_code)
        output_directory = os.path.join(input_folder_base, "Sum_Helicity_Experimental_Color", triad_code)
        if os.path.exists(input_directory):
            generate_accumulated_helicity(input_directory, output_directory)
            print(f"Generated accumulated helicity files for {lab_name}, {shift_name} shift Experimental Color, Triad {triad_code}.")
        else:
            print(f"Skipping {shift_name} shift for {lab_name} Experimental Color Triad {triad_code} as directory does not exist: {input_directory}")

# Define the root folder, suffixes, and triad codes
file_suffixes = ["medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt", "mednegra.txt"]
triad_codes = {
    (0, 2, 4): "FRT",  
    (1, 3, 5): "PLB",  
    (0, 3, 4): "FLT",  
    (1, 2, 5): "PRB",  
    (0, 2, 5): "FRB",  
    (1, 3, 4): "PLT",  
    (0, 3, 5): "FLB",  
    (1, 2, 4): "PRT",  
    (2, 4, 5): "RTB",  
    (0, 1, 3): "FLP",  
    (3, 4, 5): "LTB",  
    (0, 1, 2): "FRP"
}

# Procesar todos los laboratorios y turnos
labs = ["Lab_Betta", "Lab_Gamma"]

for lab_name in labs:
    for shift in ['V', 'M']:
        process_lab_helicity(root_folder, lab_name, shift, file_suffixes, triad_codes)

print("Processing complete. All accumulated helicity results are saved in their respective directories.")

## 9

$$

    \Huge \text{First Integral of Absolute Values of Helicity}

$$


In [ ]:
#Integral absoluta helicidad .txt
import os
import re

def process_file_absolute(file_path, output_file_path):
    # Leer las líneas del archivo
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Extraer los valores de helicidad y calcular la suma de los valores absolutos
    absolute_helicity_sum = 0
    results = []
    for line in lines:
        time_stamp, value = line.split('->')
        value = abs(float(value.strip()))  # Uso de valor absoluto
        absolute_helicity_sum += value
        results.append(f"{time_stamp.strip()} -> {absolute_helicity_sum:.10f}\n")

    # Escribir los resultados de la suma de valores absolutos en un nuevo archivo
    with open(output_file_path, 'w') as output_file:
        output_file.writelines(results)

def generate_absolute_helicity_sum(input_directory, output_directory):
    # Crear el directorio de salida si no existe
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Leer todos los archivos en el directorio especificado
    for root, dirs, files in os.walk(input_directory):
        for filename in files:
            if filename.endswith(".txt"):
                file_path = os.path.join(root, filename)
                output_filename = f"sum_absolute_{filename}"  # Cambio en el nombre del archivo de salida para reflejar los valores absolutos
                output_file_path = os.path.join(output_directory, output_filename)
                # Procesar el archivo
                process_file_absolute(file_path, output_file_path)
                # Imprimir en consola el nombre del archivo generado
                print(f"Generated file: {output_filename}")

def find_experiment_folder(root_folder, lab, shift):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder), date_regex.search(folder).group(0)
    return None, None

def process_lab_absolute_helicity(root_folder, lab_name, shift, file_suffixes, triad_codes):
    shift_name = "Vespertino" if shift == 'V' else "Matutino"
    experiment_folder, date_str = find_experiment_folder(root_folder, lab_name, shift)
    if not experiment_folder:
        print(f"No se encontró ningún directorio para {lab_name}, {shift}.")
        return

    input_folder_base = os.path.join(experiment_folder, "Data Analysis", "Processing Data")
    
    if not os.path.exists(input_folder_base):
        print(f"Skipping {shift_name} shift for {lab_name} as directory does not exist: {input_folder_base}")
        return

    # Procesar los acumulados absolutos de Baseline para cada triada
    for triad, triad_code in triad_codes.items():
        input_directory = os.path.join(input_folder_base, "Helicity_Baseline", triad_code)
        output_directory = os.path.join(input_folder_base, "Sum_Absolute_Helicity_Baseline", triad_code)
        if os.path.exists(input_directory):
            generate_absolute_helicity_sum(input_directory, output_directory)
            print(f"Generated absolute helicity sum files for {lab_name}, {shift_name} shift Baseline, Triad {triad_code}.")
        else:
            print(f"Skipping {shift_name} shift for {lab_name} Baseline Triad {triad_code} as directory does not exist: {input_directory}")

    # Procesar los acumulados absolutos de Experimental Color para cada triada
    for triad, triad_code in triad_codes.items():
        input_directory = os.path.join(input_folder_base, "Helicity_Experimental_Color", triad_code)
        output_directory = os.path.join(input_folder_base, "Sum_Absolute_Helicity_Experimental_Color", triad_code)
        if os.path.exists(input_directory):
            generate_absolute_helicity_sum(input_directory, output_directory)
            print(f"Generated absolute helicity sum files for {lab_name}, {shift_name} shift Experimental Color, Triad {triad_code}.")
        else:
            print(f"Skipping {shift_name} shift for {lab_name} Experimental Color Triad {triad_code} as directory does not exist: {input_directory}")

# Define the root folder, suffixes, and triad codes
file_suffixes = ["medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt", "mednegra.txt"]
triad_codes = {
    (0, 2, 4): "FRT",  
    (1, 3, 5): "PLB",  
    (0, 3, 4): "FLT",  
    (1, 2, 5): "PRB",  
    (0, 2, 5): "FRB",  
    (1, 3, 4): "PLT",  
    (0, 3, 5): "FLB",  
    (1, 2, 4): "PRT",  
    (2, 4, 5): "RTB",  
    (0, 1, 3): "FLP",  
    (3, 4, 5): "LTB",  
    (0, 1, 2): "FRP"
}

# Procesar todos los laboratorios y turnos
labs = ["Lab_Betta", "Lab_Gamma"]

for lab_name in labs:
    for shift in ['V', 'M']:
        process_lab_absolute_helicity(root_folder, lab_name, shift, file_suffixes, triad_codes)

print("Processing complete. All absolute helicity sum results are saved in their respective directories.")


## 10

$$

    \Huge \text{General Enstrophy File Generator}

$$


In [ ]:
#Enstrofía general .txt
import numpy as np
import os
import re

def read_vorticity_data(filepath):
    data = []
    timestamps = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])
                    vorticity_components = parts[1].strip('()').split(', ')
                    data.append([float(v) for v in vorticity_components])
        return timestamps, np.array(data)
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
        return [], np.array([])

def calculate_enstrophy(vorticities, volume):
    # Enstrophy calculation: sum of the squares of vorticity components, multiplied by volume
    enstrophy = np.sum(vorticities**2, axis=1) * volume
    return enstrophy

def format_and_save_enstrophies(timestamps, enstrophies, filename):
    try:
        with open(filename, "w") as file:
            for timestamp, enstrophy in zip(timestamps, enstrophies):
                formatted_output = f"{timestamp} -> {enstrophy:.10f}\n"
                file.write(formatted_output)
    except Exception as e:
        print(f"Error writing file {filename}: {e}")

def process_files_enstrophy(input_folder, output_folder, file_suffixes, volume):
    os.makedirs(output_folder, exist_ok=True)
    generated_files = []  # Lista para almacenar los nombres de los archivos generados

    for root, dirs, files in os.walk(input_folder):
        for filename in files:
            for file_suffix in file_suffixes:
                if filename.endswith(file_suffix):
                    base_filename = filename[:-len(file_suffix)]  # Correctly removing the suffix
                    
                    filepath_vorticity = os.path.join(root, filename)
                    
                    timestamps, vorticity = read_vorticity_data(filepath_vorticity)
                    if vorticity.size == 0:
                        continue
                    
                    enstrophy = calculate_enstrophy(vorticity, volume)
                    
                    output_filename = f"enstrophy_{base_filename}{file_suffix}"
                    output_filepath = os.path.join(output_folder, output_filename)
                    
                    format_and_save_enstrophies(timestamps, enstrophy, output_filepath)
                    generated_files.append(f"{os.path.basename(output_folder)}/{output_filename}")

    return generated_files  # Devolver la lista de archivos generados

def find_experiment_folder(root_folder, lab, shift):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder), date_regex.search(folder).group(0)
    return None, None

def process_lab_enstrophy(root_folder, lab_name, shift, file_suffixes, volume, triad_codes):
    shift_name = "Vespertino" if shift == 'V' else "Matutino"
    experiment_folder, date_str = find_experiment_folder(root_folder, lab_name, shift)
    if not experiment_folder:
        print(f"No se encontró ningún directorio para {lab_name}, {shift}.")
        return

    input_folder_base = os.path.join(experiment_folder, "Data Analysis", "Processing Data")
    
    if not os.path.exists(input_folder_base):
        print(f"Skipping {shift_name} shift for {lab_name} as directory does not exist: {input_folder_base}")
        return

    # Procesar enstrofia de Baseline para cada triada
    for triad, triad_code in triad_codes.items():
        input_directory = os.path.join(input_folder_base, "Vorticity_Baseline", triad_code)
        output_directory = os.path.join(input_folder_base, "Enstrophy_Baseline", triad_code)
        if os.path.exists(input_directory):
            generated_files = process_files_enstrophy(input_directory, output_directory, file_suffixes, volume)
            print(f"Generated enstrophy files for {lab_name}, {shift_name} shift Baseline, Triad {triad_code}:")
            for file in generated_files:
                print(f"  - {file}")
        else:
            print(f"Skipping {shift_name} shift for {lab_name} Baseline Triad {triad_code} as directory does not exist: {input_directory}")

    # Procesar enstrofia de Experimental Color para cada triada
    for triad, triad_code in triad_codes.items():
        input_directory = os.path.join(input_folder_base, "Vorticity_Experimental_Color", triad_code)
        output_directory = os.path.join(input_folder_base, "Enstrophy_Experimental_Color", triad_code)
        if os.path.exists(input_directory):
            generated_files = process_files_enstrophy(input_directory, output_directory, file_suffixes, volume)
            print(f"Generated enstrophy files for {lab_name}, {shift_name} shift Experimental Color, Triad {triad_code}:")
            for file in generated_files:
                print(f"  - {file}")
        else:
            print(f"Skipping {shift_name} shift for {lab_name} Experimental Color Triad {triad_code} as directory does not exist: {input_directory}")

# Define the root folder, suffixes, and triad codes
file_suffixes = ["medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt", "mednegra.txt"]
triad_codes = {
    (0, 2, 4): "FRT",  
    (1, 3, 5): "PLB",  
    (0, 3, 4): "FLT",  
    (1, 2, 5): "PRB",  
    (0, 2, 5): "FRB",  
    (1, 3, 4): "PLT",  
    (0, 3, 5): "FLB",  
    (1, 2, 4): "PRT",  
    (2, 4, 5): "RTB",  
    (0, 1, 3): "FLP",  
    (3, 4, 5): "LTB",  
    (0, 1, 2): "FRP"
}

# Volumes for each lab
volumes = {
    "Lab_Betta": 7.3195 , # Volume for Lab A
    "Lab_Gamma": 6.8342   # Volume for Lab B (example)
}

# Procesar todos los laboratorios y turnos
labs = ["Lab_Betta", "Lab_Gamma"]

for lab_name in labs:
    for shift in ['V', 'M']:
        process_lab_enstrophy(root_folder, lab_name, shift, file_suffixes, volumes[lab_name], triad_codes)

print("Processing complete. All enstrophy results are saved in their respective directories.")

## 11

$$

    \Huge \text{First Integral of Enstrophy}

$$


In [ ]:
#Integral Enstrofía
import os
import numpy as np
import re

def read_vorticity_data(filepath):
    data = []
    timestamps = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])
                    vorticity_components = parts[1].strip('()').split(', ')
                    data.append([float(v) for v in vorticity_components])
        return timestamps, np.array(data)
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
        return [], np.array([])

def calculate_enstrophy(vorticities, volume):
    # Enstrophy calculation: sum of the squares of vorticity components, multiplied by volume
    enstrophy = np.sum(vorticities**2, axis=1) * volume
    return enstrophy

def format_and_save_enstrophies(timestamps, enstrophies, filename):
    try:
        with open(filename, "w") as file:
            for timestamp, enstrophy in zip(timestamps, enstrophies):
                formatted_output = f"{timestamp} -> {enstrophy:.10f}\n"
                file.write(formatted_output)
    except Exception as e:
        print(f"Error writing file {filename}: {e}")

def process_files_enstrophy(input_folder, output_folder, file_suffixes, volume):
    os.makedirs(output_folder, exist_ok=True)
    generated_files = []  # Lista para almacenar los nombres de los archivos generados

    for filename in os.listdir(input_folder):
        for file_suffix in file_suffixes:
            if filename.endswith(file_suffix):
                base_filename = filename[:-len(file_suffix)]  # Correctly removing the suffix
                
                filepath_vorticity = os.path.join(input_folder, filename)
                
                timestamps, vorticity = read_vorticity_data(filepath_vorticity)
                if vorticity.size == 0:
                    continue
                
                enstrophy = calculate_enstrophy(vorticity, volume)
                
                output_filename = f"enstrophy_{base_filename}{file_suffix}"
                output_filepath = os.path.join(output_folder, output_filename)
                
                format_and_save_enstrophies(timestamps, enstrophy, output_filepath)
                generated_files.append(f"{os.path.basename(output_folder)}/{output_filename}")

    return generated_files  # Devolver la lista de archivos generados

def accumulate_enstrophy(file_path, output_file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
        
        accumulated_enstrophy = 0
        results = []
        for line in lines:
            time_stamp, value = line.split('->')
            value = float(value.strip())
            accumulated_enstrophy += value
            results.append(f"{time_stamp.strip()} -> {accumulated_enstrophy:.10f}\n")

        with open(output_file_path, 'w') as output_file:
            output_file.writelines(results)
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

def generate_accumulated_enstrophy(input_directory, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for root, dirs, files in os.walk(input_directory):
        for filename in files:
            if filename.endswith(".txt"):
                file_path = os.path.join(root, filename)
                output_filename = f"sum_{filename}"
                output_file_path = os.path.join(output_directory, output_filename)
                accumulate_enstrophy(file_path, output_file_path)
                print(f"Generated file: {output_filename}")

def find_experiment_folder(root_folder, lab, shift):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder), date_regex.search(folder).group(0)
    return None, None

def process_lab_accumulated_enstrophy(root_folder, lab_name, shift, file_suffixes, volume, triad_codes):
    shift_name = "Vespertino" if shift == 'V' else "Matutino"
    experiment_folder, date_str = find_experiment_folder(root_folder, lab_name, shift)
    if not experiment_folder:
        print(f"No se encontró ningún directorio para {lab_name}, {shift}.")
        return

    input_folder_base = os.path.join(experiment_folder, "Data Analysis", "Processing Data")
    
    if not os.path.exists(input_folder_base):
        print(f"Skipping {shift_name} shift for {lab_name} as directory does not exist: {input_folder_base}")
        return

    # Procesar enstrofia acumulada de Baseline para cada triada
    for triad, triad_code in triad_codes.items():
        input_directory = os.path.join(input_folder_base, "Enstrophy_Baseline", triad_code)
        output_directory = os.path.join(input_folder_base, "Sum_Enstrophy_Baseline", triad_code)
        if os.path.exists(input_directory):
            generate_accumulated_enstrophy(input_directory, output_directory)
            print(f"Generated accumulated enstrophy files for {lab_name}, {shift_name} shift Baseline, Triad {triad_code}.")
        else:
            print(f"Skipping {shift_name} shift for {lab_name} Baseline Triad {triad_code} as directory does not exist: {input_directory}")

    # Procesar enstrofia acumulada de Experimental Color para cada triada
    for triad, triad_code in triad_codes.items():
        input_directory = os.path.join(input_folder_base, "Enstrophy_Experimental_Color", triad_code)
        output_directory = os.path.join(input_folder_base, "Sum_Enstrophy_Experimental_Color", triad_code)
        if os.path.exists(input_directory):
            generate_accumulated_enstrophy(input_directory, output_directory)
            print(f"Generated accumulated enstrophy files for {lab_name}, {shift_name} shift Experimental Color, Triad {triad_code}.")
        else:
            print(f"Skipping {shift_name} shift for {lab_name} Experimental Color Triad {triad_code} as directory does not exist: {input_directory}")

# Define the root folder, suffixes, and triad codes
file_suffixes = ["medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt", "mednegra.txt"]
triad_codes = {
    (0, 2, 4): "FRT",  
    (1, 3, 5): "PLB",  
    (0, 3, 4): "FLT",  
    (1, 2, 5): "PRB",  
    (0, 2, 5): "FRB",  
    (1, 3, 4): "PLT",  
    (0, 3, 5): "FLB",  
    (1, 2, 4): "PRT",  
    (2, 4, 5): "RTB",  
    (0, 1, 3): "FLP",  
    (3, 4, 5): "LTB",  
    (0, 1, 2): "FRP"
}

# Volumes for each lab
volumes = {
    "Lab_Betta": 7.3195 , # Volume for Lab A
    "Lab_Gamma": 6.8342   # Volume for Lab B (example)
}

# Procesar todos los laboratorios y turnos
labs = ["Lab_Betta", "Lab_Gamma"]

for lab_name in labs:
    for shift in ['V', 'M']:
        process_lab_accumulated_enstrophy(root_folder, lab_name, shift, file_suffixes, volumes[lab_name], triad_codes)

print("Processing complete. All accumulated enstrophy results are saved in their respective directories.")

## 12

$$

    \Huge \text{First Integral Moduli Baseline vs Experimental Color}

$$


In [ ]:
import numpy as np
import os
import glob
import datetime

def read_data(filepath):
    timestamps = []
    values = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                try:
                    timestamp = datetime.datetime.strptime(parts[0], "%H:%M:%S.%f")
                    timestamps.append(timestamp)
                    values.append([float(num) for num in parts[1].strip('()').split(',')])
                except ValueError as e:
                    print(f"Error processing line in file {filepath}: {line}\n{e}")
    return timestamps, np.array(values)

def generate_combinations(baseline_files, experimental_files):
    combinations = []
    for base_file in baseline_files:
        base_num = int(base_file.split('med')[0][-2:])
        for exp_file in experimental_files:
            exp_num = int(exp_file.split('med')[0][-2:])
            if exp_num == base_num or exp_num == base_num - 1:
                combinations.append((base_file, exp_file))
            elif exp_num == base_num + 1 and (base_num + 1) <= len(baseline_files):
                combinations.append((baseline_files[base_num], exp_file))
    return combinations

def read_curl_file(file_path):
    """Reads a file containing curl components and returns a list of tuples (i, j, k)."""
    curl_components = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                try:
                    components = tuple(map(float, parts[1].strip('()').split(',')))
                    if len(components) == 3:
                        curl_components.append(components)
                except ValueError:
                    continue  # Ignore lines that cannot be converted to floats
    return curl_components

def calculate_modulus(curl_components):
    """Calculates the modulus of the curl components."""
    moduli = [np.sqrt(i**2 + j**2 + k**2) for i, j, k in curl_components]
    return moduli

def integrate_moduli(moduli):
    """Calculates the sum (integral) of the moduli."""
    return np.sum(moduli)

def compare_integrals(baseline_integral, intervention_integral):
    """Compares the integrals and reports the result."""
    difference = abs(intervention_integral - baseline_integral)
    percentage_difference = (difference / baseline_integral) * 100

    if percentage_difference > 34:
        return "There is intervention"
    elif 10 <= percentage_difference <= 33.9:
        return "Inconclusive"
    else:
        return "No evidence"

def print_result(lab, shift, baseline_file, experimental_file, baseline_integral, intervention_integral, result):
    shift_name = "Matutino" if shift == "M" else "Vespertino"
    print(f"Lab: {lab}, Shift: {shift_name}")
    print(f"Comparison Result")
    print(f"{'-'*50}")
    print(f"Baseline File: {os.path.basename(baseline_file)}")
    print(f"Experimental File: {os.path.basename(experimental_file)}")
    print(f"Baseline Integral: {baseline_integral:.2f}")
    print(f"Intervention Integral: {intervention_integral:.2f}")
    print(f"Difference: {abs(intervention_integral - baseline_integral):.2f}")
    print(f"Percentage Difference: {(abs(intervention_integral - baseline_integral) / baseline_integral) * 100:.2f}%")
    print(f"Result: {result}")
    print(f"{'='*50}\n")



def save_intervention_result(results_file, lab, shift, baseline_file, experimental_file, baseline_integral, intervention_integral, result):
    shift_name = "Matutino" if shift == "M" else "Vespertino"
    with open(results_file, 'a') as f:
        f.write(f"Lab: {lab}, Shift: {shift_name}\n")
        f.write(f"Comparison Result\n")
        f.write(f"{'-'*50}\n")
        f.write(f"Baseline File: {os.path.basename(baseline_file)}\n")
        f.write(f"Experimental File: {os.path.basename(experimental_file)}\n")
        f.write(f"Baseline Integral: {baseline_integral:.2f}\n")
        f.write(f"Intervention Integral: {intervention_integral:.2f}\n")
        f.write(f"Difference: {abs(intervention_integral - baseline_integral):.2f}\n")
        f.write(f"Percentage Difference: {(abs(intervention_integral - baseline_integral) / baseline_integral) * 100:.2f}%\n")
        f.write(f"Result: {result}\n")
        f.write(f"{'='*50}\n\n")

def main():
    results_file = os.path.join(root_folder, "intervention_results.txt")
    
    if os.path.exists(results_file):
        os.remove(results_file)  # Remove the file if it already exists

    labs = ["Lab_Betta", "Lab_Gamma"]
    shifts = ["V", "M"]

    color_map = {
        "medroja.txt": "red",
        "medmorada.txt": "purple",
        "medazul.txt": "blue",
        "medverde.txt": "green",
        "medamarilla.txt": "yellow"
    }

    for lab in labs:
        for shift in shifts:
            lab_dir = glob.glob(os.path.join(root_folder, f"*{lab}"))[0]
            date_str = os.path.basename(lab_dir).split(' ')[0]

            baseline_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Curl_Baseline")
            exp_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Curl_Experimental_Color")
            
            if not os.path.exists(baseline_dir) or not os.path.exists(exp_dir):
                print(f"Error: Directory not found for {lab} {shift}")
                continue

            baseline_files = [f for f in os.listdir(baseline_dir) if f.endswith('mednegra.txt') and "00" not in f]
            experimental_files = [f for f in os.listdir(exp_dir) if any(f.endswith(suffix) for suffix in color_map.keys()) and "00" not in f]

            combinations = generate_combinations(baseline_files, experimental_files)

            for base_file, exp_file in combinations:
                baseline_file = os.path.join(baseline_dir, base_file)
                experimental_file = os.path.join(exp_dir, exp_file)
                if os.path.exists(baseline_file) and os.path.exists(experimental_file):
                    # Reading and processing baseline file
                    baseline_curl = read_curl_file(baseline_file)
                    baseline_moduli = calculate_modulus(baseline_curl)
                    baseline_integral = integrate_moduli(baseline_moduli)

                    # Reading and processing intervention file
                    intervention_curl = read_curl_file(experimental_file)
                    intervention_moduli = calculate_modulus(intervention_curl)
                    intervention_integral = integrate_moduli(intervention_moduli)

                    # Comparing integrals
                    result = compare_integrals(baseline_integral, intervention_integral)
                    print_result(lab, shift, baseline_file, experimental_file, baseline_integral, intervention_integral, result)
                    
                    if result == "There is intervention":
                        save_intervention_result(results_file, lab, shift, baseline_file, experimental_file, baseline_integral, intervention_integral, result)

if __name__ == "__main__":
    main()

## 13

$$

    \Huge \text{Second Integral of Absolute Helicity Baseline vs Experimental Color}

$$


In [ ]:
import numpy as np
import os
import glob

def read_data(filepath):
    values = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                values.append(float(parts[1]))
    return np.array(values)

def get_sum_helicity(files, dir_path):
    sum_helicities = {"sum_absolute_helicity_1": {}, "sum_absolute_helicity_2": {}}
    for file in files:
        file_path = os.path.join(dir_path, file)
        values = read_data(file_path)
        sum_helicity = np.sum(values)
        if "sum_absolute_helicity_1" in file:
            sum_helicities["sum_absolute_helicity_1"][file] = sum_helicity
        elif "sum_absolute_helicity_2" in file:
            sum_helicities["sum_absolute_helicity_2"][file] = sum_helicity
    return sum_helicities

def generate_combinations(baseline_files, experimental_files):
    combinations = []
    for base_file in baseline_files:
        base_num = int(base_file.split('velocity_')[1].split('med')[0])
        for exp_file in experimental_files:
            exp_num = int(exp_file.split('velocity_')[1].split('med')[0])
            if exp_num == base_num or exp_num == base_num - 1:
                combinations.append((base_file, exp_file))
            elif exp_num == base_num + 1 and (base_num + 1) <= len(baseline_files):
                combinations.append((baseline_files[base_num], exp_file))
    return combinations

def calculate_percentage_difference(baseline_value, experimental_value):
    return 100 * (experimental_value - baseline_value) / baseline_value

def format_comparison_results(combinations, baseline_sums, experimental_sums, comparison_type):
    comparison_lines = [f"*** Comparison - {comparison_type} ***\n"]
    for base_file, exp_file in combinations:
        if base_file in baseline_sums[comparison_type] and exp_file in experimental_sums[comparison_type]:
            baseline_value = baseline_sums[comparison_type][base_file]
            experimental_value = experimental_sums[comparison_type][exp_file]
            percentage_diff = calculate_percentage_difference(baseline_value, experimental_value)
            comparison_lines.append(f"Baseline: {base_file} - {baseline_value:.2f}\n")
            comparison_lines.append(f"Experimental: {exp_file} - {experimental_value:.2f}\n")
            comparison_lines.append(f"*** Percentage Difference: {percentage_diff:.2f}% ***\n\n")
    return comparison_lines

def format_statistical_summary(baseline_sums, experimental_sums):
    summary_lines = ["*** Statistical Summary ***\n"]
    for helicity_type in baseline_sums:
        baseline_values = list(baseline_sums[helicity_type].values())
        experimental_values = list(experimental_sums[helicity_type].values())

        # Verifica si las listas están vacías
        if not baseline_values:
            summary_lines.append(f"{helicity_type} Baseline: No data\n")
        else:
            max_baseline_value = max(baseline_values)
            min_baseline_value = min(baseline_values)
            max_baseline_file = [k for k, v in baseline_sums[helicity_type].items() if v == max_baseline_value][0]
            min_baseline_file = [k for k, v in baseline_sums[helicity_type].items() if v == min_baseline_value][0]
            summary_lines.append(f"{helicity_type}:\n")
            summary_lines.append(f"  Baseline - Mean: {np.mean(baseline_values):.2f}, Std Dev: {np.std(baseline_values):.2f}\n")
            summary_lines.append(f"    Max: {max_baseline_value:.2f} ({max_baseline_file})\n")
            summary_lines.append(f"    Min: {min_baseline_value:.2f} ({min_baseline_file})\n")

        # Verifica si las listas están vacías para datos experimentales
        if not experimental_values:
            summary_lines.append(f"{helicity_type} Experimental: No data\n")
        else:
            max_experimental_value = max(experimental_values)
            min_experimental_value = min(experimental_values)
            max_experimental_file = [k for k, v in experimental_sums[helicity_type].items() if v == max_experimental_value][0]
            min_experimental_file = [k for k, v in experimental_sums[helicity_type].items() if v == min_experimental_value][0]
            summary_lines.append(f"  Experimental - Mean: {np.mean(experimental_values):.2f}, Std Dev: {np.std(experimental_values):.2f}\n")
            summary_lines.append(f"    Max: {max_experimental_value:.2f} ({max_experimental_file})\n")
            summary_lines.append(f"    Min: {min_experimental_value:.2f} ({min_experimental_file})\n")

        summary_lines.append("\n")

    return summary_lines

def main():
    root_folder = "path/to/root_folder"  # Define la ruta de la carpeta raíz
    labs = ["Lab_Betta", "Lab_Gamma"]
    shifts = ["V", "M"]

    results = []

    for lab in labs:
        for shift in shifts:
            lab_dirs = glob.glob(os.path.join(root_folder, f"*{lab}"))
            if not lab_dirs:
                print(f"No se encontró ningún directorio para {lab}.")
                continue
            lab_dir = lab_dirs[0]
            date_str = os.path.basename(lab_dir).split(' ')[0]
            baseline_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Sum_Absolute_Helicity_Baseline")
            exp_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Sum_Absolute_Helicity_Experimental_Color")

            if not os.path.exists(baseline_dir) or not os.path.exists(exp_dir):
                print(f"Omitiendo cálculo para {lab} {shift} ya que el directorio no existe.")
                continue

            baseline_files = [f for f in os.listdir(baseline_dir) if f.endswith('mednegra.txt') and "00" not in f]
            experimental_files = [f for f in os.listdir(exp_dir) if f.endswith('.txt') and "00" not in f]

            baseline_sums = get_sum_helicity(baseline_files, baseline_dir)
            experimental_sums = get_sum_helicity(experimental_files, exp_dir)

            header = f"\n\n*** Resultados para {lab} - Turno: {'Vespertino' if shift == 'V' else 'Matutino'} - Fecha: {date_str} ***\n"
            results.append(header)
            results.append("=" * len(header) + "\n\n")

            # Generar combinaciones y formatear resultados de comparación
            combinations_1 = generate_combinations(baseline_files, experimental_files)
            results.extend(format_comparison_results(combinations_1, baseline_sums, experimental_sums, "sum_absolute_helicity_1"))
            results.extend(format_comparison_results(combinations_1, baseline_sums, experimental_sums, "sum_absolute_helicity_2"))

            # Añadir resumen estadístico
            results.append("*** Statistical Summary ***\n")
            results.extend(format_statistical_summary(baseline_sums, experimental_sums))
            results.append("=" * len(header) + "\n\n")

    output_filepath = os.path.join(root_folder, "SUM TOTAL OF HELICITY BY TRIAD COMPONENTS.txt")
    with open(output_filepath, 'w') as output_file:
        output_file.writelines(results)

    print(f"Resultados guardados en {output_filepath}")

    # Imprimir resultados en consola
    for line in results:
        print(line, end='')

if __name__ == "__main__":
    main()

In [10]:
import os
import re

# Función para decodificar el texto hexadecimal
def decodificar_hex(hex_string):
    try:
        hex_cleaned = hex_string.strip().replace(' ', '')
        decoded_string = bytes.fromhex(hex_cleaned).decode('ascii', errors='ignore').strip()
        return decoded_string
    except Exception as e:
        print(f"Error decodificando: {e}")
        return None

# Función para procesar el archivo y extraer la hora y el valor limpio
def procesar_archivo_para_hora_y_texto(ruta_archivo):
    resultados = []
    
    with open(ruta_archivo, 'r') as archivo:
        for linea in archivo:
            match_hora = re.search(r'(\d{2}-\d{2}-\d{4} \d{2}:\d{2}:\d{2}\.\d{3})', linea)
            if match_hora:
                hora = match_hora.group(1).split(' ')[1]  # Obtener solo la parte de la hora
                partes = linea.strip().split(',')
                if len(partes) > 3:
                    hex_data = partes[-1]
                    decoded_value = decodificar_hex(hex_data)

                    if decoded_value:
                        resultados.append(f"{hora}, {decoded_value}")
                    else:
                        resultados.append(f"{hora}, .")
    
    return resultados

# Función para guardar los resultados en un archivo de salida
def guardar_resultados(ruta_salida, resultados):
    with open(ruta_salida, 'w') as archivo_salida:
        for resultado in resultados:
            archivo_salida.write(resultado + '\n')

# Función para construir la ruta del archivo a procesar
def construir_ruta(root_folder, fecha, lab, turno, color, numero):
    colorin = color.lower()
    return os.path.join(root_folder, f"{fecha} - Lab_{lab}", f"{fecha}.{turno} - Lab_{lab}", f"{color}", "pesajes", f"{numero:02d}med{colorin}-pesajes.log")

# Función para filtrar resultados
def filtrar_resultados(resultados):
    # Expresión regular para el formato deseado
    pattern = r'^\d{2}:\d{2}:\d{2}\.\d{3}, \d+\.\d{2}g$'
    return [resultado for resultado in resultados if re.match(pattern, resultado.strip())]

# Si este es el archivo principal
if __name__ == "__main__":
    root_folder = r"F:\Datos\16Sep24"
    fecha = extraer_fecha(root_folder)  # Asegúrate de definir esta función
    labs = ["Betta", "Gamma"]
    turnos = ["M", "V"]
    colores = ["Amarilla", "Roja", "Azul", "Verde", "Morada"]
    numeros_archivo = range(1, 11)

    for lab in labs:
        for turno in turnos:
            for color in colores:
                for numero in numeros_archivo:
                    ruta_archivo = construir_ruta(root_folder, fecha, lab, turno, color, numero)
                    
                    if os.path.exists(ruta_archivo):
                        ruta_salida = os.path.join(os.path.dirname(ruta_archivo), f"resultados_{numero:02d}.txt")

                        print(f"\nProcesando para Lab: {lab}, Turno: {turno}, Color: {color}, Archivo: {numero:02d}")
                        print(f"Archivo a procesar: {ruta_archivo}")
                        
                        # Procesar el archivo para extraer hora y texto limpio
                        resultados = procesar_archivo_para_hora_y_texto(ruta_archivo)

                        # Filtrar resultados
                        resultados_filtrados = filtrar_resultados(resultados)

                        # Guardar los resultados filtrados en el archivo de salida
                        guardar_resultados(ruta_salida, resultados_filtrados)
                    else:
                        print(f"Archivo no encontrado: {ruta_archivo}")
    print("\nProcesamiento completo.")


Procesando para Lab: Betta, Turno: M, Color: Amarilla, Archivo: 01
Archivo a procesar: F:\Datos\16Sep24\16Sep24 - Lab_Betta\16Sep24.M - Lab_Betta\Amarilla\pesajes\01medamarilla-pesajes.log
Error decodificando: non-hexadecimal number found in fromhex() arg at position 0

Procesando para Lab: Betta, Turno: M, Color: Amarilla, Archivo: 02
Archivo a procesar: F:\Datos\16Sep24\16Sep24 - Lab_Betta\16Sep24.M - Lab_Betta\Amarilla\pesajes\02medamarilla-pesajes.log
Error decodificando: non-hexadecimal number found in fromhex() arg at position 0

Procesando para Lab: Betta, Turno: M, Color: Amarilla, Archivo: 03
Archivo a procesar: F:\Datos\16Sep24\16Sep24 - Lab_Betta\16Sep24.M - Lab_Betta\Amarilla\pesajes\03medamarilla-pesajes.log
Error decodificando: non-hexadecimal number found in fromhex() arg at position 0
Archivo no encontrado: F:\Datos\16Sep24\16Sep24 - Lab_Betta\16Sep24.M - Lab_Betta\Amarilla\pesajes\04medamarilla-pesajes.log
Archivo no encontrado: F:\Datos\16Sep24\16Sep24 - Lab_Betta\16

In [3]:
import os
import re
import pandas as pd

# Función para extraer la fecha del nombre del directorio raíz
def extraer_fecha(root_folder):
    nombre_directorio = os.path.basename(root_folder)
    match = re.search(r'(\d{2}[A-Za-z]{3}\d{2})', nombre_directorio)
    
    if match:
        fecha_str = match.group(1)  # Captura la fecha en formato '16Sep24'
        return fecha_str
    else:
        raise ValueError(f"No se encontró una fecha válida en el nombre del directorio {root_folder}.")

# Función para cargar y procesar los datos
def cargar_y_procesar_datos(ruta_archivo):
    datos_procesados = []
    
    with open(ruta_archivo, 'r') as archivo:
        for linea in archivo:
            partes = linea.split(',')
            hora = partes[0].strip()  # Extrae la hora
            valor = partes[1].strip()  # Extrae el valor con 'g'
            
            # Elimina la 'g' y convierte a float
            valor_numero = float(valor.replace('g', ''))
            # Aplica el tratamiento Dato * 9.81 / 5.00
            valor_procesado = valor_numero * 9.81 / 5.00
            
            # Guarda la hora y el valor procesado
            datos_procesados.append(f"{hora}, {valor_procesado:.2f}\n")
    
    return datos_procesados

# Función para guardar los datos procesados en un archivo .txt
def guardar_datos_txt(datos_procesados, ruta_salida):
    with open(ruta_salida, 'w') as archivo_salida:
        archivo_salida.writelines(datos_procesados)
    print(f"Datos procesados guardados en: {ruta_salida}")

# Función principal para procesar y guardar archivos
def procesar_archivos(root_folder):
    labs = ["Betta", "Gamma"]
    turnos = ["M", "V"]
    colores = ["Amarilla", "Roja", "Azul", "Verde", "Morada"]
    numeros_archivo = range(1, 11)

    fecha = extraer_fecha(root_folder)

    for lab in labs:
        for turno in turnos:
            for color in colores:
                for numero in numeros_archivo:
                    colorin = color.lower()
                    ruta_archivo = os.path.join(root_folder, f"{fecha} - Lab_{lab}", f"{fecha}.{turno} - Lab_{lab}", "Pesajes", f"0{numero}med{colorin}.txt")
                    
                    if os.path.exists(ruta_archivo):
                        # Cargar y procesar los datos del archivo
                        datos_procesados = cargar_y_procesar_datos(ruta_archivo)

                        # Definir la carpeta y el nombre del archivo de salida
                        output_folder = os.path.join(root_folder, f"{fecha} - Lab_{lab}", f"{fecha}.{turno} - Lab_{lab}", "pesajes", "procesados")
                        os.makedirs(output_folder, exist_ok=True)
                        ruta_salida = os.path.join(output_folder, f"resultados_0{numero}_{colorin}.txt")

                        # Guardar los datos procesados en un archivo .txt
                        guardar_datos_txt(datos_procesados, ruta_salida)
                    else:
                        print(f"Archivo no encontrado: {ruta_archivo}")

# Si este es el archivo principal
if __name__ == "__main__":
    procesar_archivos(root_folder)
    print("\nProcesamiento completo.")

Archivo no encontrado: F:\Datos\16Sep24\16Sep24 - Lab_Betta\16Sep24.M - Lab_Betta\Pesajes\01medamarilla.txt
Archivo no encontrado: F:\Datos\16Sep24\16Sep24 - Lab_Betta\16Sep24.M - Lab_Betta\Pesajes\02medamarilla.txt
Archivo no encontrado: F:\Datos\16Sep24\16Sep24 - Lab_Betta\16Sep24.M - Lab_Betta\Pesajes\03medamarilla.txt
Archivo no encontrado: F:\Datos\16Sep24\16Sep24 - Lab_Betta\16Sep24.M - Lab_Betta\Pesajes\04medamarilla.txt
Archivo no encontrado: F:\Datos\16Sep24\16Sep24 - Lab_Betta\16Sep24.M - Lab_Betta\Pesajes\05medamarilla.txt
Archivo no encontrado: F:\Datos\16Sep24\16Sep24 - Lab_Betta\16Sep24.M - Lab_Betta\Pesajes\06medamarilla.txt
Archivo no encontrado: F:\Datos\16Sep24\16Sep24 - Lab_Betta\16Sep24.M - Lab_Betta\Pesajes\07medamarilla.txt
Archivo no encontrado: F:\Datos\16Sep24\16Sep24 - Lab_Betta\16Sep24.M - Lab_Betta\Pesajes\08medamarilla.txt
Archivo no encontrado: F:\Datos\16Sep24\16Sep24 - Lab_Betta\16Sep24.M - Lab_Betta\Pesajes\09medamarilla.txt
Archivo no encontrado: F:\Da